In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path="DataAllMining_Imputed.csv"#"final_data.csv"
df=pd.read_csv(path)

In [3]:
df.columns

Index(['Unnamed: 0', 'cat__Departamento_AMAZONAS',
       'cat__Departamento_ANTIOQUIA', 'cat__Departamento_ARAUCA',
       'cat__Departamento_ATLANTICO', 'cat__Departamento_BOGOTA',
       'cat__Departamento_BOLIVAR', 'cat__Departamento_BOYACA',
       'cat__Departamento_CALDAS', 'cat__Departamento_CAQUETA',
       'cat__Departamento_CASANARE', 'cat__Departamento_CAUCA',
       'cat__Departamento_CESAR', 'cat__Departamento_CHOCO',
       'cat__Departamento_CORDOBA', 'cat__Departamento_CUNDINAMARCA',
       'cat__Departamento_GUAINIA', 'cat__Departamento_GUAVIARE',
       'cat__Departamento_HUILA', 'cat__Departamento_LA GUAJIRA',
       'cat__Departamento_MAGDALENA', 'cat__Departamento_META',
       'cat__Departamento_NARINO', 'cat__Departamento_NORTE DE SANTANDER',
       'cat__Departamento_PUTUMAYO', 'cat__Departamento_QUINDIO',
       'cat__Departamento_RISARALDA',
       'cat__Departamento_SAN ANDRES PROVIDENCIA',
       'cat__Departamento_SANTANDER', 'cat__Departamento_SUCRE',
   

In [4]:
df=df[['num__Year',
       'num__Month/Day', 'num__Rain_day', 'num__Rain_month', 'num__Temp_day',
       'num__Temp_month', 'num__Humi_day', 'num__Humi_month', 'Departamento']]

In [5]:
df=df.rename(columns={'num__Year':'Year',
                      'num__Month/Day':'Month/Day', 
                      'num__Rain_day':'Rain_day', 
                      'num__Rain_month':'Rain_month', 
                      'num__Temp_day':'Temp_day',
                      'num__Temp_month':'Temp_month', 
                      'num__Humi_day':'Humi_day', 
                      'num__Humi_month':'Humi_month'})

In [6]:
df

,Year,Month/Day,Rain_day,Rain_month,Temp_day,Temp_month,Humi_day,Humi_month,Departamento
0,2006.0,1.0,0.0,0.000,25.017378,23.900927,88.102616,81.407159,AMAZONAS
1,2006.0,1.0,0.0,199.500,19.324113,18.957285,72.291080,74.619183,ANTIOQUIA
2,2006.0,1.0,0.0,0.000,4.905264,3.951957,78.035940,74.718325,ARAUCA
3,2006.0,1.0,0.0,0.000,25.899578,25.022157,77.344941,74.797651,ATLANTICO
4,2006.0,1.0,0.1,56.600,13.195833,13.873947,72.581196,73.826251,BOGOTA
...,...,...,...,...,...,...,...,...,...
7519,2024.0,12.0,31.5,2549.600,28.600000,27.541524,78.865939,83.827022,SUCRE
7520,2024.0,12.0,7.7,5263.420,25.547887,21.558212,65.350509,78.336104,TOLIMA
7521,2024.0,12.0,5.0,2829.232,25.485417,23.484995,71.907145,79.920617,VALLE DEL CAUCA
7522,2024.0,12.0,0.0,305.400,27.176081,25.577988,82.720172,81.038751,VAUPES


# hindcast leave one year

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample


def hindcast_leave_one_year_long(
        data_path: str | Path,
        target_col: str = "Rain_month",
        target_month: int = 2,
        feature_month: int = 1,
        random_state: int = 0,
    ):
    """
    Hindcast evaluation for the *long* table structure:
        Year | Month/Day | <var>_day | <var>_month | Departamento

    Parameters
    ----------
    data_path     : CSV file with the structure shown in the screenshot.
    target_col    : One of {'Rain_month','Temp_month','Humi_month'}.
    target_month  : Month index (2‑12) to be predicted.
    feature_month : Month whose *_day variables are used as predictors (default=1, January).
    random_state  : Reproducibility for tree‑based models.

    Returns
    -------
    per_year_df   : RMSE per fold per model (DataFrame).
    summary_df    : Global RMSE, R² and skill vs. climatology (DataFrame).
    """

    # ------------------------------------------------------------------
    # 1) Read data and reshape to wide format
    # ------------------------------------------------------------------
    df = pd.read_csv(data_path)
    df=df[['num__Year',
       'num__Month/Day', 'num__Rain_day', 'num__Rain_month', 'num__Temp_day',
       'num__Temp_month', 'num__Humi_day', 'num__Humi_month', 'Departamento']]
    df=df.rename(columns={'num__Year':'Year',
                      'num__Month/Day':'Month/Day', 
                      'num__Rain_day':'Rain_day', 
                      'num__Rain_month':'Rain_month', 
                      'num__Temp_day':'Temp_day',
                      'num__Temp_month':'Temp_month', 
                      'num__Humi_day':'Humi_day', 
                      'num__Humi_month':'Humi_month'})
    
    required_cols = {"Year", "Month/Day", "Departamento",
                     "Rain_day", "Temp_day", "Humi_day",
                     "Rain_month", "Temp_month", "Humi_month"}
    missing = required_cols.difference(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    # ------------------------------------------------------------------
    # 2) Build feature table (January *_day columns)
    # ------------------------------------------------------------------
    feats = (df.loc[df["Month/Day"] == feature_month,
                    ["Year", "Departamento",
                     "Rain_day", "Temp_day", "Humi_day"]]
               .rename(columns={
                   "Rain_day": f"Jan_Rain_day",
                   "Temp_day": f"Jan_Temp_day",
                   "Humi_day": f"Jan_Humi_day"}))

    # ------------------------------------------------------------------
    # 3) Build target table (chosen month & variable)
    # ------------------------------------------------------------------
    targs = (df.loc[df["Month/Day"] == target_month,
                    ["Year", "Departamento", target_col]]
               .rename(columns={target_col: "Target"}))

    # ------------------------------------------------------------------
    # 4) Merge and clean
    # ------------------------------------------------------------------
    data = (pd.merge(feats, targs, on=["Year", "Departamento"], how="inner")
              .dropna())                              # drop incomplete records

    X_cols = ["Jan_Rain_day", "Jan_Temp_day", "Jan_Humi_day"]
    X      = data[X_cols].values
    y      = data["Target"].values
    groups = data["Year"].astype(int).values          # one fold == one year

    # ------------------------------------------------------------------
    # 5) Define models
    # ------------------------------------------------------------------
    models = {
        "Baseline": None,
        "Ridge": make_pipeline(StandardScaler(), Ridge(alpha=1.0)),
        "RF": RandomForestRegressor(
                  n_estimators=300, max_depth=6, n_jobs=-1,
                  min_samples_leaf=2, random_state=random_state),
        "HGB": HistGradientBoostingRegressor(
                  max_depth=3, learning_rate=0.1,
                  l2_regularization=0.0, random_state=random_state),
    }

    # ------------------------------------------------------------------
    # 6) Leave‑One‑Year‑Out cross‑validation
    # ------------------------------------------------------------------
    logo = LeaveOneGroupOut()
    yhat_all = {m: np.empty_like(y, dtype=float) for m in models}
    rows = []

    for train_idx, test_idx in logo.split(X, y, groups):
        yr = groups[test_idx][0]
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Baseline climatology
        yhat_base = np.full_like(y_test, y_train.mean())
        yhat_all["Baseline"][test_idx] = yhat_base

        # Other models
        for name, mdl in models.items():
            if name == "Baseline":
                continue
            mdl.fit(X_train, y_train)
            yhat = mdl.predict(X_test)
            yhat_all[name][test_idx] = yhat

        # Fold metrics
        for name in models:
            rmse = np.sqrt(mean_squared_error(
                       y_test, yhat_all[name][test_idx]))
            rows.append({"Year": yr, "Model": name,
                         "Fold_RMSE": rmse, "n_obs": len(test_idx)})

    per_year_df = (pd.DataFrame(rows)
                     .pivot(index="Year", columns="Model", values="Fold_RMSE")
                     .sort_index())

    # ------------------------------------------------------------------
    # 7) Global metrics
    # ------------------------------------------------------------------
    baseline_rmse = np.sqrt(mean_squared_error(y, yhat_all["Baseline"]))
    summary = []
    for name in models:
        rmse = np.sqrt(mean_squared_error(y, yhat_all[name]))
        r2   = r2_score(y, yhat_all[name])
        skill = 1 - rmse / baseline_rmse
        summary.append({"Model": name,
                        "Global_RMSE": rmse, "R²": r2, "Skill_vs_Base": skill})
    summary_df = pd.DataFrame(summary).set_index("Model")

    return per_year_df, summary_df, y, yhat_all, groups

## Calculations...

In [8]:
# ----------------------------------------------------------------------
# Predict February total rainfall using January daily means
# ----------------------------------------------------------------------
per_year_rain, summary_rain, y_rain, yhat_all_rain, groups_rain = hindcast_leave_one_year_long(
    data_path="DataAllMining_Imputed.csv",          # ← update path
    target_col="Rain_month",                # what you want to predict
    target_month=2,                         # 2 = February
    feature_month=1                         # 1 = January
)

In [9]:
print("\n=== Per‑Year RMSE: RAIN ===")
per_year_rain.round(2)


=== Per‑Year RMSE: RAIN ===


Model,Baseline,HGB,RF,Ridge
Year,,,,
2006,1635.25,877.39,740.91,581.71
2007,1725.51,5853.46,2834.32,546.00
2008,2193.75,2557.80,1812.52,1854.43
2009,1547.18,1076.17,298.41,336.42
2010,1582.73,1894.12,1011.40,639.72
2011,1552.46,2251.03,727.54,548.68
2012,13419.75,13653.75,13622.36,13584.70
2013,1540.44,1940.97,725.91,946.47
2014,1528.53,1579.87,771.58,833.51


In [10]:
print("\n=== Global Metrics: RAIN ===")
summary_rain.round(3)


=== Global Metrics: RAIN ===


,Global_RMSE,R²,Skill_vs_Base
Model,,,
Baseline,12920.524,-0.005,0.000
Ridge,8350.719,0.580,0.354
RF,11348.874,0.225,0.122
HGB,12392.975,0.076,0.041


In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

def _dm_test_yearly(e1_yearly, e2_yearly, h=1, alternative="two-sided"):
    """
    Diebold–Mariano for yearly-aggregated losses.
    For h=1 (your case), this is a paired t-test on d_t = e1_t - e2_t.
    """
    e1 = np.asarray(e1_yearly, dtype=float)
    e2 = np.asarray(e2_yearly, dtype=float)
    d  = e1 - e2                      # loss differential
    T  = d.size
    dbar = d.mean()

    if h <= 1:
        # paired t-test (no serial correlation across years)
        s2 = d.var(ddof=1)
        stat = dbar / np.sqrt(s2 / T)
        # p-value with Student-t df=T-1 (falls back to normal if SciPy not present)
        try:
            from scipy.stats import t as student_t
            if alternative == "two-sided":
                p = 2 * (1 - student_t.cdf(abs(stat), df=T-1))
            elif alternative == "greater":
                p = 1 - student_t.cdf(stat, df=T-1)
            else:  # "less"
                p = student_t.cdf(stat, df=T-1)
        except Exception:
            # normal approx
            from math import erf, sqrt
            z = abs(stat)
            p_two = 2 * (1 - 0.5 * (1 + erf(z / np.sqrt(2))))
            if alternative == "two-sided":
                p = p_two
            elif alternative == "greater":
                p = p_two / 2 if stat > 0 else 1 - p_two / 2
            else:
                p = p_two / 2 if stat < 0 else 1 - p_two / 2
        return stat, p

    # Optional: HAC variance (Newey–West) if you ever use multi-step horizon h>1
    d_center = d - dbar
    Tfloat = float(T)
    gamma0 = (d_center @ d_center) / Tfloat
    var = gamma0
    for k in range(1, h):
        w = 1 - k / h
        gamma_k = (d_center[k:] @ d_center[:-k]) / Tfloat
        var += 2 * w * gamma_k
    stat = dbar / np.sqrt(var / Tfloat)
    # normal approx for HAC case
    from math import erf, sqrt
    p = 2 * (1 - 0.5 * (1 + erf(abs(stat) / np.sqrt(2))))
    return stat, p


def more_test(y, yhat_all, groups, model_name="Ridge", baseline_name="Baseline",
              n_boot=10_000, random_state=0):
    """
    y           : array (n_samples,)
    yhat_all    : dict with model predictions, e.g. {"Ridge": yhat, "Baseline": yhat, ...}
    groups      : array of Year per sample (same order as y)
    """

    y          = np.asarray(y, dtype=float)
    yhat_model = np.asarray(yhat_all[model_name], dtype=float)
    yhat_base  = np.asarray(yhat_all[baseline_name], dtype=float)
    groups     = np.asarray(groups)

    # máscara de NaNs por seguridad
    m = (~np.isnan(y)) & (~np.isnan(yhat_model)) & (~np.isnan(yhat_base))
    y, yhat_model, yhat_base, groups = y[m], yhat_model[m], yhat_base[m], groups[m]

    # ============= 1) Bootstrap por años: IC del RMSE-skill =============
    rng = np.random.default_rng(random_state)
    years = np.unique(groups)
    skills = []
    for _ in range(n_boot):
        yrs = rng.choice(years, size=len(years), replace=True)
        idx = np.isin(groups, yrs)
        rmse_m = mean_squared_error(y[idx], yhat_model[idx])
        rmse_b = mean_squared_error(y[idx], yhat_base [idx])
        skills.append(1 - rmse_m / rmse_b)
    ci_low, ci_high = np.percentile(skills, [2.5, 97.5])
    print(f"{model_name} RMSE-skill 95% CI: {ci_low:.3f} – {ci_high:.3f}")

    # ============= 2) DM (t pareado sobre diferencias anuales de MSE) =============
    e_m = (yhat_model - y)**2
    e_b = (yhat_base  - y)**2
    yearly = (pd.DataFrame({"Year": groups, "e_m": e_m, "e_b": e_b})
                .groupby("Year").mean())
    dm_stat, dm_p = _dm_test_yearly(yearly["e_m"].values, yearly["e_b"].values, h=1)
    print(f"DM statistic = {dm_stat:.3f}, p-value = {dm_p:.4f}")

    return (ci_low, ci_high), (dm_stat, dm_p)


In [12]:
print("\n=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: RAIN ===")
(ci_low, ci_high), (dm_stat, dm_p) = more_test(
    y_rain, yhat_all_rain, groups_rain,
    model_name="Ridge", baseline_name="Baseline",
    n_boot=10_000, random_state=42)


=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: RAIN ===
Ridge RMSE-skill 95% CI: -0.240 – 0.716
DM statistic = -0.973, p-value = 0.3437


## Calculations...

In [13]:
# ----------------------------------------------------------------------
# Predict February total temperature using January daily means
# ----------------------------------------------------------------------
per_year_temp, summary_temp, y_temp, yhat_all_temp, groups_temp = hindcast_leave_one_year_long(
        data_path="DataAllMining_Imputed.csv",          # ← update path
        target_col="Temp_month",                # what you want to predict
        target_month=2,                         # 2 = February
        feature_month=1                         # 1 = January
    )

In [14]:
print("\n=== Per‑Year RMSE: TEMP ===")
per_year_temp.round(2)


=== Per‑Year RMSE: TEMP ===


Model,Baseline,HGB,RF,Ridge
Year,,,,
2006,7.10,7.57,7.14,6.95
2007,5.37,1.14,0.98,2.46
2008,5.81,1.40,1.52,2.20
2009,6.21,2.15,2.83,1.19
2010,6.39,2.01,2.40,1.80
2011,6.83,3.49,3.71,3.57
2012,7.73,5.72,5.64,5.88
2013,6.48,3.42,3.46,3.11
2014,7.34,2.17,2.47,2.08


In [15]:
print("\n=== Global Metrics: TEMP ===")
summary_temp.round(3)


=== Global Metrics: TEMP ===


,Global_RMSE,R²,Skill_vs_Base
Model,,,
Baseline,5.972,-0.003,0.000
Ridge,2.896,0.764,0.515
RF,3.046,0.739,0.490
HGB,2.979,0.750,0.501


In [16]:
print("\n=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: TEMP ===")
(ci_low, ci_high), (dm_stat, dm_p) = more_test(
    y_temp, yhat_all_temp, groups_temp,
    model_name="Ridge", baseline_name="Baseline",
    n_boot=10_000, random_state=42)


=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: TEMP ===
Ridge RMSE-skill 95% CI: 0.681 – 0.876
DM statistic = -11.082, p-value = 0.0000


## Calculations...

In [17]:
# ----------------------------------------------------------------------
# Predict February total humidity using January daily means
# ----------------------------------------------------------------------
per_year_humi, summary_humi, y_humi, yhat_all_humi, groups_humi = hindcast_leave_one_year_long(
        data_path="DataAllMining_Imputed.csv",          # ← update path
        target_col="Humi_month",                # what you want to predict
        target_month=2,                         # 2 = February
        feature_month=1                         # 1 = January
    )

In [18]:
print("\n=== Per‑Year RMSE: HUMI ===")
per_year_humi.round(2)


=== Per‑Year RMSE: HUMI ===


Model,Baseline,HGB,RF,Ridge
Year,,,,
2006,8.11,7.67,7.59,6.63
2007,5.99,4.74,4.75,4.82
2008,8.41,5.59,6.62,6.02
2009,7.15,4.95,4.06,4.37
2010,12.10,13.03,8.88,9.31
2011,14.20,8.73,6.11,7.67
2012,16.49,9.33,6.03,8.33
2013,11.99,8.42,6.02,7.36
2014,12.99,9.23,8.76,10.19


In [19]:
print("\n=== Global Metrics: HUMI ===")
summary_humi.round(3)


=== Global Metrics: HUMI ===


,Global_RMSE,R²,Skill_vs_Base
Model,,,
Baseline,9.428,-0.006,0.000
Ridge,7.107,0.428,0.246
RF,6.774,0.480,0.281
HGB,7.478,0.367,0.207


In [20]:
print("\n=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: HUMI ===")
(ci_low, ci_high), (dm_stat, dm_p) = more_test(
    y_humi, yhat_all_humi, groups_humi,
    model_name="Ridge", baseline_name="Baseline",
    n_boot=10_000, random_state=42)


=== 1) Bootstrap por años: IC del RMSE-skill & 2) DM (t pareado sobre diferencias anuales de MSE) =============: HUMI ===
Ridge RMSE-skill 95% CI: 0.261 – 0.550
DM statistic = -2.777, p-value = 0.0124


# Pipeline: 
- Climatología por departamento (LOY‑O)
- Persistencia (mes m−1 del mismo año y departamento)
- calcular skill y significancia (IC bootstrap + DM) usando tu more_test, y
- producir tablas resumen para discusión.

In [25]:
# ===========================================
# 0) CONFIG
# ===========================================
DATA_PATH = "DataAllMining_Imputed.csv"   # 
TARGET_VARS = ["Rain_month", "Temp_month", "Humi_month"]
MONTHS = list(range(2, 13))                    # 2..12
FEATURE_MONTH = 1                               # enero

# ===========================================
# 1) Helpers: reproducen la tabla "data" del hindcast
#    y construyen baselines extra
# ===========================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def _load_long(path):
    df = pd.read_csv(path)
    # quitar espacios accidentales
    df.columns = df.columns.str.strip()

    # renombrar SOLO las que existan (soporta ambos esquemas)
    rename_map = {
        'num__Year':        'Year',
        'num__Month/Day':   'Month/Day',
        'num__Rain_day':    'Rain_day',
        'num__Rain_month':  'Rain_month',
        'num__Temp_day':    'Temp_day',
        'num__Temp_month':  'Temp_month',
        'num__Humi_day':    'Humi_day',
        'num__Humi_month':  'Humi_month',
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # chequeo y normalización mínima de tipos
    required = ['Year','Month/Day','Rain_day','Temp_day','Humi_day',
                'Rain_month','Temp_month','Humi_month','Departamento']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Columns missing after standardization: {missing}. "
                       f"Available: {list(df.columns)}")

    # tipos numéricos coherentes para los filtros por mes/año
    df['Year'] = df['Year'].astype(int)
    df['Month/Day'] = df['Month/Day'].astype(int)

    return df

def build_sample_table(path, target_col, target_month, feature_month=1):
    """Replica exactamente el 'data' interno del hindcast y devuelve (data, df_long)."""
    df = _load_long(path)
    feats = (df.loc[df["Month/Day"] == feature_month,
                    ["Year","Departamento","Rain_day","Temp_day","Humi_day"]]
               .rename(columns={"Rain_day":"Jan_Rain_day",
                                "Temp_day":"Jan_Temp_day",
                                "Humi_day":"Jan_Humi_day"}))
    targs = (df.loc[df["Month/Day"] == target_month,
                    ["Year","Departamento",target_col]]
               .rename(columns={target_col:"Target"}))
    data = pd.merge(feats, targs, on=["Year","Departamento"], how="inner").dropna()
    return data, df

def dept_climatology_baseline(data):
    """Climatología por departamento con LOY-O: media de otros años del mismo depto."""
    g = data.groupby("Departamento")["Target"].agg(["sum","count"])
    tmp = data.join(g, on="Departamento")
    # (suma - valor del año actual) / (n - 1)
    base = (tmp["sum"] - tmp["Target"]) / (tmp["count"] - 1)
    # si algún depto tuviera un solo año (raro), cae en NaN -> usa media global
    return base.fillna(data["Target"].mean()).to_numpy()

def persistence_baseline(df_long, data, target_col, target_month):
    """Persistencia: valor del mes m-1 del mismo año y departamento."""
    prev_m = target_month - 1
    prev = (df_long[df_long["Month/Day"] == prev_m]
            [["Year","Departamento", target_col]]
            .rename(columns={target_col:"Persist"}))
    out = pd.merge(data[["Year","Departamento"]], prev, on=["Year","Departamento"], how="left")
    return out["Persist"].to_numpy()

# ===========================================
# 2) Loop: meses 2–12, variables; añade baselines y resume
# ===========================================
all_rows = []

for var in TARGET_VARS:
    for m in MONTHS:
        # 2.1 Hindcast con tus funciones
        per_year, summary, y, yhat_all, groups = hindcast_leave_one_year_long(
            DATA_PATH, target_col=var, target_month=m, feature_month=FEATURE_MONTH
        )

        # 2.2 Reconstruye tabla 'data' y añade baselines extra
        data_tbl, df_long = build_sample_table(DATA_PATH, var, m, FEATURE_MONTH)
        # chequeo de alineación (opcional)
        assert np.allclose(y, data_tbl["Target"].to_numpy(), equal_nan=True)

        base_dept = dept_climatology_baseline(data_tbl)
        base_pers = persistence_baseline(df_long, data_tbl, var, m)

        # extiende yhat_all para usar more_test sin tocar tu función
        yhat_all_ext = dict(yhat_all)
        yhat_all_ext["DeptClim"] = base_dept
        yhat_all_ext["Persist"]  = base_pers

        # 2.3 Métricas globales y significancia (bootstrap + DM) para Ridge
        for baseline_name in ["Baseline", "DeptClim", "Persist"]:
            # skill y RMSE globales para los tres modelos
            for model_name in ["Ridge","RF","HGB"]:
                rmse_m = mean_squared_error(y, yhat_all_ext[model_name])
                rmse_b = mean_squared_error(y, yhat_all_ext[baseline_name])
                skill  = 1 - rmse_m / rmse_b
                row = {"Variable": var.replace("_month","").upper(),
                       "Month": m,
                       "Baseline": baseline_name,
                       "Model": model_name,
                       "RMSE_model": rmse_m,
                       "RMSE_baseline": rmse_b,
                       "Skill": skill,
                       "R2": r2_score(y, yhat_all_ext[model_name])}
                # IC y DM solo para Ridge (reduce costo)
                if model_name == "Ridge":
                    (ci_low, ci_high), (dm_stat, dm_p) = more_test(
                        y, yhat_all_ext, groups,
                        model_name="Ridge", baseline_name=baseline_name,
                        n_boot=5_000,         # puedes subir a 10_000
                        random_state=0
                    )
                    row.update({"CI_low": ci_low, "CI_high": ci_high,
                                "DM_stat": dm_stat, "DM_p": dm_p})
                all_rows.append(row)

# DataFrame maestro con TODOS los resultados
results_df = pd.DataFrame(all_rows)

# ===========================================
# 3) Tablas resumen útiles para discusión
# ===========================================

# 3.1 Pivot: Skill de RIDGE por variable, mes y baseline
ridge_pivot = (results_df[results_df["Model"]=="Ridge"]
               .pivot_table(index=["Variable","Baseline"],
                            columns="Month", values="Skill", aggfunc="mean")
               .sort_index())

# 3.2 Resumen estadístico por variable y baseline (RIDGE)
ridge_summary = (results_df[results_df["Model"]=="Ridge"]
                 .groupby(["Variable","Baseline"])
                 .agg(Mean_Skill=("Skill","mean"),
                      Median_Skill=("Skill","median"),
                      Min_Skill=("Skill","min"),
                      Max_Skill=("Skill","max"),
                      Months_Positive=("Skill", lambda s: int((s>0).sum())),
                      Months_Total=("Skill","size"))
                 .reset_index())

# 3.3 Conteo de meses con significancia (p<0.05) para RIDGE
sig_counts = (results_df[(results_df["Model"]=="Ridge") &
                         (results_df["DM_p"].notna())]
              .assign(sig=lambda d: d["DM_p"]<0.05)
              .groupby(["Variable","Baseline"])["sig"]
              .agg(Significant_Months="sum").reset_index())

# 3.4 Comparativa rápida de modelos (skill medio vs. baseline DeptClim)
models_vs_dept = (results_df[results_df["Baseline"]=="DeptClim"]
                  .groupby(["Variable","Model"])
                  .agg(Mean_Skill=("Skill","mean"),
                       Median_Skill=("Skill","median"))
                  .reset_index()
                  .sort_values(["Variable","Mean_Skill"], ascending=[True,False]))

# 3.5 Guarda a disco
results_df.to_csv("hindcast_full_results.csv", index=False)
ridge_pivot.to_csv("hindcast_ridge_skill_pivot.csv")
ridge_summary.to_csv("hindcast_ridge_skill_summary.csv", index=False)
sig_counts.to_csv("hindcast_ridge_significance_counts.csv", index=False)
models_vs_dept.to_csv("hindcast_models_vs_deptclim.csv", index=False)

# 3.6 Muestras legibles en pantalla
print("\n=== RIDGE Skill by Baseline (rows) and Month (cols) ===")
print(ridge_pivot.round(3))

print("\n=== RIDGE Summary (mean/median/min/max skill, count of positive months) ===")
print(ridge_summary.round(3))

print("\n=== RIDGE: number of months with DM p<0.05 ===")
print(sig_counts)

print("\n=== Model comparison vs Department Climatology (mean skill) ===")
print(models_vs_dept.round(3))


Ridge RMSE-skill 95% CI: -0.246 – 0.716
DM statistic = -0.973, p-value = 0.3437
Ridge RMSE-skill 95% CI: -0.091 – 0.791
DM statistic = -1.047, p-value = 0.3089
Ridge RMSE-skill 95% CI: -26.369 – 0.035
DM statistic = 1.012, p-value = 0.3251
Ridge RMSE-skill 95% CI: 0.046 – 0.603
DM statistic = -1.309, p-value = 0.2071
Ridge RMSE-skill 95% CI: 0.040 – 0.594
DM statistic = -1.310, p-value = 0.2068
Ridge RMSE-skill 95% CI: -9.007 – 0.944
DM statistic = -0.866, p-value = 0.3977
Ridge RMSE-skill 95% CI: -11.057 – 0.299
DM statistic = 0.908, p-value = 0.3757
Ridge RMSE-skill 95% CI: -11.125 – 0.394
DM statistic = 0.917, p-value = 0.3713
Ridge RMSE-skill 95% CI: -5.681 – 0.755
DM statistic = 0.906, p-value = 0.3767
Ridge RMSE-skill 95% CI: -44.303 – 0.411
DM statistic = 0.954, p-value = 0.3529
Ridge RMSE-skill 95% CI: -30.086 – 0.613
DM statistic = 0.941, p-value = 0.3590
Ridge RMSE-skill 95% CI: -214.905 – -0.964
DM statistic = 1.278, p-value = 0.2173
Ridge RMSE-skill 95% CI: -1.261 – 0.619
D

In [23]:
df

,Year,Month/Day,Rain_day,Rain_month,Temp_day,Temp_month,Humi_day,Humi_month,Departamento
0,2006.0,1.0,0.0,0.000,25.017378,23.900927,88.102616,81.407159,AMAZONAS
1,2006.0,1.0,0.0,199.500,19.324113,18.957285,72.291080,74.619183,ANTIOQUIA
2,2006.0,1.0,0.0,0.000,4.905264,3.951957,78.035940,74.718325,ARAUCA
3,2006.0,1.0,0.0,0.000,25.899578,25.022157,77.344941,74.797651,ATLANTICO
4,2006.0,1.0,0.1,56.600,13.195833,13.873947,72.581196,73.826251,BOGOTA
...,...,...,...,...,...,...,...,...,...
7519,2024.0,12.0,31.5,2549.600,28.600000,27.541524,78.865939,83.827022,SUCRE
7520,2024.0,12.0,7.7,5263.420,25.547887,21.558212,65.350509,78.336104,TOLIMA
7521,2024.0,12.0,5.0,2829.232,25.485417,23.484995,71.907145,79.920617,VALLE DEL CAUCA
7522,2024.0,12.0,0.0,305.400,27.176081,25.577988,82.720172,81.038751,VAUPES


In [28]:
per_year_temp.round(2).to_excel("algo1.xlsx")
per_year_rain.round(2).to_excel("algo2.xlsx")
per_year_humi.round(2).to_excel("algo3.xlsx")